In [184]:
from Crypto.Cipher import AES
from Crypto.Cipher import Blowfish
from Crypto import Random
from struct import pack
import random
import string
from bitstring import BitArray

In [185]:
#Here i build the 1-bit-different messages

MSG1=[]
MSG2=[]

for i in range (msgCount):
    msg=""
    for j in range (15):
        msg=msg+random.choice(string.ascii_letters)    
        
                         #all the letters are the same, I only change the last letter to have 1-bit difference
    message1=msg+"b"     # b = 01100010, ends with 0
    message2=msg+"c"     # c = 01100011, ends with 1, so we have one-bit difference
    MSG1.append(message1)     
    MSG2.append(message2)
    #print(message1 + "   " + message2)        #for testing purposes, uncomment to print the 1-bit-different messages


In [186]:
# AES is based on XOR operation between key and plaintext.
# In AES, we need the same length on both the message and the key.

def AES_ECB(msgCount,key):

    BitsChanged=[]   #Here I will store the number of bits that were different in the same position on every pair of ciphertexts

    for i in range (msgCount):
        ECB = AES.new(key, AES.MODE_ECB)
        message1 = MSG1[i]
        message2 = MSG2[i]

        ciphertext1 = ECB.encrypt(message1)
        ciphertext2 = ECB.encrypt(message2)

        c1 = BitArray(ciphertext1)
        c2 = BitArray(ciphertext2)
        #print(c1.bin)     # for testing purposes, uncomment to see the 2 binary ciphertexts
        #print(c2.bin)
        #print("\n")

        counter=0
        for j in range (128):      # Here I count the different bits in the ciphertext
            if c1.bin[j]!=c2.bin[j]:
                counter=counter + 1 
        BitsChanged.append(counter)


    average=0         #Here i find the average bit difference
    for i in BitsChanged:
        average=average+i
    average=average//msgCount

    print("AES - ECB Mode\n______________\n")
    print("Different bits in every pair of ciphertexts:\n")    
    print(BitsChanged)
    print("\nAverage bit difference:\n")  
    print(str(average) + " bits, " + str("{:.2f}".format(average/128*100)) + "% chance of bit-difference")
    print("\nSince it's ~50%, we can safely assume that the AES.ECB has a good Avalance Effect\n")


In [187]:
def AES_CBC(msgCount,key,IV):

    BitsChanged=[]   #Here I will store the number of bits that were different in the same position on every pair of ciphertexts

    for i in range (msgCount):
        CBC = AES.new(key, AES.MODE_CBC, IV)
        message1 = MSG1[i]
        message2 = MSG2[i]

        ciphertext1 = CBC.encrypt(message1)
        ciphertext2 = CBC.encrypt(message2)

        c1 = BitArray(ciphertext1)
        c2 = BitArray(ciphertext2)
        #print(c1.bin)     # for testing purposes, uncomment to see the 2 binary ciphertexts
        #print(c2.bin)
        #print("\n")

        counter=0
        for j in range (128):      # Here I count the different bits in the ciphertext
            if c1.bin[j]!=c2.bin[j]:
                counter=counter + 1 
        BitsChanged.append(counter)


    average=0         #Here i find the average bit difference
    for i in BitsChanged:
        average=average+i
    average=average//msgCount

    print("AES - CBC Mode\n______________\n")
    print("Different bits in every pair of ciphertexts:\n")    
    print(BitsChanged)
    print("\nAverage bit difference:\n")  
    print(str(average) + " bits, " + str("{:.2f}".format(average/128*100)) + "% chance of bit-difference")
    print("\nSince it's ~50%, we can safely assume that the AES.CBC has a good Avalance Effect\n")


In [188]:
#Blowfish is a symmetric block cipher.
#It has a fixed data block size of 8 bytes and its keys can vary in length from 32 to 448 bits (4 to 56 bytes).
#Its keys should be chosen to be big enough to withstand a brute force attack (e.g. at least 16 bytes).

def Blowfish_CBC(msgCount,key):
    
    BitsChanged=[]   #Here I will store the number of bits that were different in the same position on every pair of ciphertexts
    
    for i in range (msgCount):
        bs = Blowfish.block_size
        key = key
        iv = Random.new().read(bs)
        CBC = Blowfish.new(key, Blowfish.MODE_CBC, iv)

        message1 = MSG1[i].encode('utf-8')
        plen = bs - divmod(len(message1),bs)[1]
        padding = [plen]*plen
        padding = pack('b'*plen, *padding)
        ciphertext1 = iv + CBC.encrypt(message1 + padding)

        message2 = MSG2[i].encode('utf-8')
        plen = bs - divmod(len(message2),bs)[1]
        padding = [plen]*plen
        padding = pack('b'*plen, *padding)
        ciphertext2 = iv + CBC.encrypt(message2 + padding)
        
        c1 = BitArray(ciphertext1)
        c2 = BitArray(ciphertext2)
        #print(c1.bin)     # for testing purposes, uncomment to see the 2 binary ciphertexts
        #print(c2.bin)
        #print("\n")
        
        counter=0
        for j in range (c1.length):      # Here I count the different bits in the ciphertext
            if c1.bin[j]!=c2.bin[j]:
                counter=counter + 1 
        BitsChanged.append(counter)


    average=0         #Here i find the average bit difference
    for i in BitsChanged:
        average=average+i
    average=average//msgCount
    
    print("Blowfish - CBC Mode\n______________\n")
    print("Different bits in every pair of ciphertexts:\n")    
    print(BitsChanged)
    print("\nAverage bit difference:\n")  
    print(str(average) + " bits, " + str("{:.2f}".format(average/256*100)) + "% chance of bit-difference")
    print("\nSince it is NOT ~50%, we can safely assume that the Blowfish.CBC does NOT have good Avalance Effect\n")


In [189]:
msgCount = 35     # Number of messages pairs
key = 'The 16 digit key'
IV16 = 'This is my 16 IV'


AES_ECB(msgCount,key)
print("----------------------------------------------------------------------------------------------\n")
AES_CBC(msgCount,key,IV16)
print("----------------------------------------------------------------------------------------------\n")
Blowfish_CBC(msgCount,key)

AES - ECB Mode
______________

Different bits in every pair of ciphertexts:

[68, 73, 71, 75, 72, 80, 67, 66, 67, 61, 60, 64, 69, 61, 61, 66, 66, 63, 69, 59, 79, 64, 59, 65, 59, 62, 65, 72, 71, 63, 70, 64, 70, 70, 59]

Average bit difference:

66 bits, 51.56% chance of bit-difference

Since it's ~50%, we can safely assume that the AES.ECB has a good Avalance Effect

----------------------------------------------------------------------------------------------

AES - CBC Mode
______________

Different bits in every pair of ciphertexts:

[63, 66, 63, 64, 64, 69, 69, 65, 62, 55, 61, 57, 65, 74, 70, 63, 58, 69, 65, 65, 63, 59, 79, 65, 73, 69, 66, 69, 65, 61, 79, 68, 69, 62, 68]

Average bit difference:

65 bits, 50.78% chance of bit-difference

Since it's ~50%, we can safely assume that the AES.CBC has a good Avalance Effect

----------------------------------------------------------------------------------------------

Blowfish - CBC Mode
______________

Different bits in every pair of ci